In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Load data file
df = spark.read.format("csv").option("header","true").load("Files/Drug_Consumption_Quantified.csv")
display(df)

StatementMeta(, ebf02c2c-a868-4ec4-9ac6-35f89cb7ecb8, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6234bc7c-3bc9-46d3-8d4b-e1c94a7086ea)

In [6]:
df = sdf.toPandas()
# remove CL prefix and coveryt type to int
target_cols = ['Alcohol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 
               'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 
               'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 
               'Mushrooms', 'Nicotine', 'VSA']

for col in target_cols:
    df[col] = df[col].str.replace('CL', '').astype(int)

StatementMeta(, 0e04def5-66e8-4469-b43f-0e68efc5de09, 8, Finished, Available, Finished)

In [4]:
# Select 2 out of 18 output features, then predict each selected feature by using a multiclass classification 
# model. 
# X and targets y
X = df[['Age', 'Gender', 'Education', 'Country', 'Ethnicity',
        'Nscore', 'Escore', 'Oscore', 'AScore', 'Cscore', 'Impulsive', 'SS']]

#remove "CL" prefixes (CL0, CL1 >> 0, 1)
df['Cannabis'] = df['Cannabis'].astype(str).str.replace('CL', '').astype(int)
df['Coke']     = df['Coke'].astype(str).str.replace('CL', '').astype(int)

y_cannabis = df['Cannabis']
y_coke = df['Coke']

# Train and test splits
X_train_ca, X_test_ca, y_train_ca, y_test_ca = train_test_split(X, y_cannabis, test_size=0.2, random_state=42)
X_train_coke, X_test_coke, y_train_coke, y_test_coke = train_test_split(X, y_coke, test_size=0.2, random_state=42)

# Scaling cannabis=ca
scaler = StandardScaler()
X_train_ca_scaled = scaler.fit_transform(X_train_ca)
X_test_ca_scaled = scaler.transform(X_test_ca)
# Scaling coke
scaler_coke = StandardScaler()
X_train_coke_scaled = scaler_coke.fit_transform(X_train_coke)
X_test_coke_scaled = scaler_coke.transform(X_test_coke)


# Model 1: Logistic Regression (Cannabis) 
lr_ca = LogisticRegression(max_iter=500, multi_class='multinomial')
lr_ca.fit(X_train_ca_scaled, y_train_ca)
y_pred_lr_ca = lr_ca.predict(X_test_ca_scaled)
acc_lr_ca = accuracy_score(y_test_ca, y_pred_lr_ca)
#Logistic Regression (Coke)
lr_coke = LogisticRegression(max_iter=500, multi_class='multinomial')
lr_coke.fit(X_train_coke_scaled,y_train_coke)
y_pred_lr_coke= lr_coke.predict(X_test_coke_scaled)
acc_lr_coke = accuracy_score(y_test_coke,y_pred_lr_coke)

#Model 2:  Random Forest (Cannabis)  (scaling is not necessary)
rf_ca = RandomForestClassifier(random_state=42)
rf_ca.fit(X_train_ca, y_train_ca)
y_pred_rf_ca = rf_ca.predict(X_test_ca)
acc_rf_ca = accuracy_score(y_test_ca, y_pred_rf_ca)

# Random Forest (Coke)
rf_coke = RandomForestClassifier(random_state=42)
rf_coke.fit(X_train_coke,y_train_coke)
y_pred_rf_coke=rf_coke.predict(X_test_coke)
acc_rf_coke=accuracy_score(y_test_coke,y_pred_rf_coke)

print(f"Accuracy (Logistic Regression - Cannabis): {acc_lr_ca:.3f}")
print(f"Accuracy (Logistic Regression - Coke): {acc_lr_coke:.3f}")
print(f"Accuracy (Random Forest - Cannabis): {acc_rf_ca:.3f}")
print(f"Accuracy (Random Forest - Coke): {acc_rf_coke:.3f}")


StatementMeta(, 0e04def5-66e8-4469-b43f-0e68efc5de09, 6, Finished, Available, Finished)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy (Logistic Regression - Cannabis): 0.427
Accuracy (Logistic Regression - Coke): 0.576
Accuracy (Random Forest - Cannabis): 0.398
Accuracy (Random Forest - Coke): 0.586
